# Werkcollege-opdrachten Week 1.3

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [3]:
import pandas 
import sqlite3
import warnings
warnings.simplefilter('ignore')

Zet het bestand go_sales_train.sqlite in een makkelijk te vinden map

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales_train.sqlite te maken.

In [5]:

conn = sqlite3.connect('project/sql/go_sales_train2.sqlite')
print("SQLite3 Connection: ",conn)



SQLite3 Connection:  <sqlite3.Connection object at 0x106da07c0>


Lees van de ingelezen go_sales_train-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- returned_item
- return_reason

In [3]:
tabellen = [
    "product", "product_type", "product_line", "sales_staff", 
    "sales_branch", "retailer_site", "country", "order_header", 
    "order_details", "returned_item", "return_reason"
]

dataframes = {}

for tabel in tabellen:
    dataframes[tabel] = pandas.read_sql_query(f"SELECT * FROM {tabel}", conn)
    print(f"Tabel '{tabel}' ingelezen met {len(dataframes[tabel])} rijen.")



Tabel 'product' ingelezen met 115 rijen.
Tabel 'product_type' ingelezen met 21 rijen.
Tabel 'product_line' ingelezen met 5 rijen.
Tabel 'sales_staff' ingelezen met 102 rijen.
Tabel 'sales_branch' ingelezen met 28 rijen.
Tabel 'retailer_site' ingelezen met 391 rijen.
Tabel 'country' ingelezen met 21 rijen.
Tabel 'order_header' ingelezen met 4784 rijen.
Tabel 'order_details' ingelezen met 37757 rijen.
Tabel 'returned_item' ingelezen met 619 rijen.
Tabel 'return_reason' ingelezen met 5 rijen.


Krijg je een "no such table" error? Dan heb je misschien met .connect() per ongeluk een leeg  databasebestand (.sqlite) aangemaakt. <u>Let op:</u> lees eventueel de informatie uit het Notebook van werkcollege 1.1b nog eens goed door.

Als je tijdens onderstaande opdrachten uit het oog verliest welke tabellen er allemaal zijn, kan je deze Pythoncode uitvoeren:

In [4]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in
pandas.read_sql(sql_query, conn)
#Op de puntjes hoort de connectie naar go_sales_train óf go_staff_train óf go_crm_train te staan.

,name
0,country
1,order_details
2,order_header
3,order_method
4,product
5,product_line
6,product_type
7,retailer_site
8,return_reason
9,returned_item


erachter 

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [5]:
filtered_producten = dataframes["product"][['PRODUCT_NAME','PRODUCTION_COST']]
filtered_producten = filtered_producten[
    (filtered_producten["PRODUCTION_COST"] > 50) & (filtered_producten["PRODUCTION_COST"] < 100)
]
print(filtered_producten)

                        PRODUCT_NAME  PRODUCTION_COST
5          TrailChef Deluxe Cook Set            85.11
7             TrailChef Double Flame            75.00
16                   Hibernator Lite            60.00
17                        Hibernator            86.00
19   Hibernator Self - Inflating Mat            78.55
22               Hibernator Camp Cot            65.33
23      Canyon Mule Climber Backpack            62.50
45           Granite Climbing Helmet            52.86
47             Husky Harness Extreme            55.00
57                    Granite Hammer            57.13
68                         Polar Ice            73.33
69                      Polar Sports            80.00
70                        Polar Wave            60.00
71                     Polar Extreme            72.50
74                      Edge Extreme            80.00
77                         Seeker 35            79.19
78                         Seeker 50            92.58
79                    Seeker

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [134]:
filtered_marges = dataframes["product"][["PRODUCT_NAME","MARGIN"]]
filtered_marges = filtered_marges[
    (filtered_marges["MARGIN"] > 0.60) | (filtered_marges["MARGIN"] < 0.20)
]
print(filtered_marges)

                    PRODUCT_NAME  MARGIN
20                Hibernator Pad    0.17
23  Canyon Mule Climber Backpack    0.17
85             BugShield Natural    0.67
86               BugShield Spray    0.67
87         BugShield Lotion Lite    0.70
88              BugShield Lotion    0.63
89             BugShield Extreme    0.63


Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [135]:
filter_francs = dataframes["country"][
    dataframes["country"]["CURRENCY_NAME"] == "francs"
][["COUNTRY"]].sort_values("COUNTRY")

print(filter_francs)


        COUNTRY
15      Belgium
0        France
7   Switzerland


Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [136]:
filter_introdatum_marge_meer_dan_vijftig = dataframes["product"][
    dataframes["product"]["MARGIN"] > 0.50
][["INTRODUCTION_DATE"]].drop_duplicates()

print(filter_introdatum_marge_meer_dan_vijftig)

    INTRODUCTION_DATE
66         2000-10-26
85         1995-02-15
101        2003-12-10
103        2003-12-18
104        2003-12-27
106        2004-01-13
110        2003-12-15


Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 7 rijen)

In [137]:
filter_verkoopafdeling = dataframes["retailer_site"].loc[
    (dataframes["retailer_site"]["ADDRESS2"].notnull()) & 
    (dataframes["retailer_site"]["REGION"].notnull()),
    ["ADDRESS1", "CITY"]
].drop_duplicates()

print(filter_verkoopafdeling)
#ik begrijp niet hoe je hier 7 rijen als uitkomst kan hebben, heb ook database gecheckt en er zijn wel meer dan 7 rijen die een address1 en region aanwezig hebben.

                  ADDRESS1        CITY
13      500, Place d'Armes    Montréal
31        Röntgenstraße 90   Frankfurt
59         Grubesallee 141     Hamburg
63       2479 Broad Street    Hartford
74      10423 Kenwood Road  Cincinnati
..                     ...         ...
350           Albany House    Brighton
351           Bridge Court    Brighton
354         Exchange Court    Brighton
355          Keyford Court  Nottingham
357  Vanwall Business Park     Reading

[78 rows x 2 columns]


Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [138]:
filter_landen_dollar = dataframes["country"].loc[
    (dataframes["country"]["CURRENCY_NAME"] == "dollars") | (dataframes["country"]["CURRENCY_NAME"] == "new dollar"),
    ["COUNTRY"]
]
print(filter_landen_dollar)

          COUNTRY
2   United States
3          Canada
11         Taiwan
14      Australia


Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [139]:
filter_beide_adressen_filliaal = dataframes["retailer_site"].loc[
    (dataframes["retailer_site"]["POSTAL_ZONE"].str.startswith("D", na=False)) &
    (dataframes["retailer_site"]["ADDRESS2"].notnull()),
    ["ADDRESS1","ADDRESS2","CITY"]
].sort_values(by="ADDRESS2")

print(filter_beide_adressen_filliaal)

            ADDRESS1       ADDRESS2       CITY
31  Röntgenstraße 90  3. Tür rechts  Frankfurt
59   Grubesallee 141       4. Stock    Hamburg


## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [140]:
return_count = dataframes["returned_item"]["RETURN_QUANTITY"].sum()

print(return_count)

11908


Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [141]:
aantal_regios = dataframes["retailer_site"]["REGION"].nunique()

print(aantal_regios)

110


Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [258]:
hoogste = dataframes["product"]["MARGIN"].max()
laagste = dataframes["product"]["MARGIN"].min()
gem = dataframes["product"]["MARGIN"].mean()

print(f"Hoogste winstmarge: {hoogste}")
print(f"laagste winstmarge: {laagste}")
print(f"Gemiddelde winstmarge: {gem.round(2)}")

Hoogste winstmarge: 0.7
laagste winstmarge: 0.17
Gemiddelde winstmarge: 0.4


Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [260]:
count_vestiging = dataframes["retailer_site"][dataframes["retailer_site"]["ADDRESS2"].isnull()].shape[0]
print(count_vestiging)

286


Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [144]:
gem_kostprijs = dataframes["order_details"][
    (dataframes)["order_details"]["UNIT_SALE_PRICE"] < (dataframes)["order_details"]["UNIT_PRICE"]
]["UNIT_COST"].mean()

print(gem_kostprijs)

127.34087927598804


Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [145]:
medewerkers_per_functie = dataframes["sales_staff"].groupby("POSITION_EN").size().reset_index(name="AANTAL MEDEWERKERS")

print(medewerkers_per_functie)

                    POSITION_EN  AANTAL MEDEWERKERS
0                Branch Manager                  19
1              District Manager                   4
2               General Manager                   1
3  Level 1 Sales Representative                  13
4  Level 2 Sales Representative                  39
5  Level 3 Sales Representative                  24
6              Regional Manager                   2


Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [146]:
medewerkers_per_telefoon = dataframes["sales_staff"].groupby("WORK_PHONE").size().reset_index(name="AANTAL_MEDEWERKERS")

medewerkers_per_telefoon_filtered = medewerkers_per_telefoon[medewerkers_per_telefoon["AANTAL_MEDEWERKERS"] > 4]

print(medewerkers_per_telefoon_filtered)

             WORK_PHONE  AANTAL_MEDEWERKERS
4     +(44) 121 3505267                   5
10  +(61) 03 2982  4242                   5
11  +31 (0)20 692 93 94                   5
13    +33 1 68 94 52 20                   5
18     1 (206) 292-0012                   5
19     1 (305) 557-4810                   6
20     1 (310) 281-5722                   5
21     1 (403) 232-5986                   5
22     1 (416) 493-5595                   5
23     1 (617) 268-6754                   6


## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [147]:
adres_filliaal = dataframes["retailer_site"].merge(dataframes["country"], on="COUNTRY_CODE")
adres_ned_filliaal = adres_filliaal[adres_filliaal["COUNTRY"] == "Netherlands"]
resultaat = adres_ned_filliaal[["ADDRESS1","CITY"]]
print(resultaat)

                      ADDRESS1          CITY
240                   Beets 36         Beets
241               Perestraat 1      Deventer
242  Laan van Meerdervoort 966      Den Haag
243             Kalfjeslaan 47    Amstelveen
244               Startbaan 11    Amstelveen
245      Noorderspoorsingel 28     Groningen
246              Hogehilweg 13     Amsterdam
247             Spoorstraat 43    Varsseveld
248             Westhavenweg 8     Amsterdam
249              Westzeedijk 4        Muiden
250            Gooiergracht 91         Laren
251            Hoofdstraat 150   Pieterburen
252                Olde Diek 2         Laren
253     Amsterdamsestraatweg 8      Maarssen
254        Krayenhoffstraat 11     Amsterdam
255         Overschiestraat 19     Amsterdam
256              Havenbuurt 12        Marken
257               Duivendaal 1    Wageningen
258              Voorstraat 37  Nieuweschans
259         Stadhouderslaan 18     Rotterdam


Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [148]:
merge_product_type_en_product = dataframes["product"].merge(dataframes["product_type"], on="PRODUCT_TYPE_CODE")
eyewear_producten = merge_product_type_en_product[merge_product_type_en_product["PRODUCT_TYPE_EN"] == "Eyewear"]
eyewear_producten = eyewear_producten[["PRODUCT_NAME"]]

print(eyewear_producten)

     PRODUCT_NAME
67      Polar Sun
68      Polar Ice
69   Polar Sports
70     Polar Wave
71  Polar Extreme


Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [149]:
merge_sales_branch_met_sales_staff = dataframes["sales_branch"].merge(dataframes["sales_staff"], on="SALES_BRANCH_CODE")
overzicht_adres_naam = merge_sales_branch_met_sales_staff[merge_sales_branch_met_sales_staff["POSITION_EN"] == "Branch Manager"][["ADDRESS1","FIRST_NAME","LAST_NAME"]].drop_duplicates()

print(overzicht_adres_naam)


                            ADDRESS1 FIRST_NAME      LAST_NAME
0      75, rue du Faubourg St-Honoré      Denis           Pagé
5                    Piazza Duomo, 1      Maria      Iacobucci
10               Singelgravenplein 4       Kick        Kalkman
14                    Schwabentor 35      Frank          Fuchs
17                  Leopoldstraße 36      Fritz         Hirsch
22               Isafjordsgatan 30 C      Karin      Bergström
25      7800, 756 - 6th Avenue. S.W.      Sally          White
28                  789 Yonge Street    Georges  Saint-Germain
33            1288 Dorchester Avenue       Alex      Rodriguez
39                   299 Yale Avenue   Samantha          Floyd
44        1288 South Barrington Ave.      Corey         Wright
50                    10032 NW 186th      Julie          Olsen
55               6c, rue de l'Église      Julie        Béchard
59  Prol. Paseo de la Reforma No. 51      Lucía          Reyna
66                  2315 Queen's Ave      Alice        

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 7 rijen) 

In [153]:
merge_order_header_met_sales_staff = dataframes["order_header"].merge(dataframes["sales_staff"], on="SALES_STAFF_CODE")
managers = merge_order_header_met_sales_staff[merge_order_header_met_sales_staff["POSITION_EN"].str.contains(" Manager")][["POSITION_EN","ORDER_DATE"]]

print(managers)

         POSITION_EN  ORDER_DATE
3185  Branch Manager  2024-09-09
3186  Branch Manager  2024-03-13
3187  Branch Manager  2024-01-09


Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [157]:
merge_order_details_met_product = dataframes["product"].merge(dataframes["order_details"],on="PRODUCT_NUMBER")
merged_order_details_product_en_product_name = merge_order_details_met_product.merge(dataframes["product_type"],on="PRODUCT_TYPE_CODE")

merged_order_details_product_en_product_name = merged_order_details_product_en_product_name[merged_order_details_product_en_product_name["QUANTITY"] > 750][["PRODUCT_NAME","PRODUCT_TYPE_EN"]].drop_duplicates()

print(merged_order_details_product_en_product_name)

                PRODUCT_NAME    PRODUCT_TYPE_EN
5427                Star Peg              Tents
11452              Firefly 2           Lanterns
28608        BugShield Spray  Insect Repellents
28892  BugShield Lotion Lite  Insect Repellents
29430       BugShield Lotion  Insect Repellents
29442      BugShield Extreme  Insect Repellents
30741         Sun Shelter 15          Sunscreen
31164         Sun Shelter 30          Sunscreen
31775             Sun Shield          Sunscreen


Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [165]:
filter_korting = (merge_order_details_met_product["UNIT_PRICE"] - merge_order_details_met_product["UNIT_SALE_PRICE"]) / merge_order_details_met_product["UNIT_PRICE"] > 0.40
filter_korting_resultaat = merge_order_details_met_product.loc[filter_korting, ["PRODUCT_NAME"]].drop_duplicates()

print(filter_korting_resultaat)


                       PRODUCT_NAME
27830             BugShield Natural
28283               BugShield Spray
28694         BugShield Lotion Lite
29127              BugShield Lotion
29464             BugShield Extreme
30562                Sun Shelter 15
30929                Sun Shelter 30
34441  Hailstorm Titanium Woods Set


Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

In [173]:
merge_order_details_met_returned_item = dataframes["order_details"].merge(dataframes["returned_item"],on="ORDER_DETAIL_CODE")

merge_met_product = merge_order_details_met_returned_item.merge(
    dataframes["return_reason"], on="RETURN_REASON_CODE"
)

filter_returned_item_meer_dan_90_procent = merge_met_product [ 
    merge_met_product["RETURN_QUANTITY"] / merge_met_product["QUANTITY"] > 0.90
]

resultaat90 = filter_returned_item_meer_dan_90_procent[["RETURN_DESCRIPTION_EN"]].drop_duplicates()

print(resultaat90)

    RETURN_DESCRIPTION_EN
0   Wrong product ordered
4   Wrong product shipped
5  Unsatisfactory product


## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [176]:
aantal_producten_per_type = merge_product_type_en_product.groupby("PRODUCT_TYPE_EN")["PRODUCT_NUMBER"].count().reset_index()

aantal_producten_per_type.columns = ["Product Type", "Aantal Producten"]


print(aantal_producten_per_type)

            Product Type  Aantal Producten
0             Binoculars                 4
1   Climbing Accessories                 7
2           Cooking Gear                10
3                Eyewear                 5
4              First Aid                 5
5       Golf Accessories                 4
6      Insect Repellents                 5
7                  Irons                 4
8                 Knives                 5
9               Lanterns                12
10            Navigation                 4
11                 Packs                 6
12               Putters                 3
13                  Rope                 4
14                Safety                 4
15         Sleeping Bags                 7
16             Sunscreen                 5
17                 Tents                 6
18                 Tools                 6
19               Watches                 5
20                 Woods                 4


Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [179]:
merge_retailer_site_met_country = dataframes["retailer_site"].merge(dataframes["country"], on="COUNTRY_CODE")

aantal_vestiging_per_land = merge_retailer_site_met_country.groupby("COUNTRY")["RETAILER_SITE_CODE"].count().reset_index()

aantal_vestiging_per_land.columns = ["Land","Aantal vestigingen"]

print(aantal_vestiging_per_land)


              Land  Aantal vestigingen
0        Australia                  10
1          Austria                  10
2          Belgium                  10
3           Brazil                   1
4           Canada                  31
5            China                  10
6          Denmark                   6
7          Finland                   5
8           France                  29
9          Germany                  32
10           Italy                  12
11           Japan                  25
12           Korea                  10
13          Mexico                  10
14     Netherlands                  20
15           Spain                   9
16          Sweden                  12
17     Switzerland                  13
18          Taiwan                  10
19  United Kingdom                  25
20   United States                 101


Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [188]:
merge_order_details_met_product_met_product_type = merge_order_details_met_product.merge(dataframes["product_type"],on="PRODUCT_TYPE_CODE")

cooking_gear = merge_order_details_met_product_met_product_type[merge_order_details_met_product_met_product_type["PRODUCT_TYPE_EN"] == "Cooking Gear"]

cooking_gear_hvlheid_en_gem_verkoopprijs = cooking_gear.groupby("PRODUCT_NAME").agg(
    totaal_verkocht = ("QUANTITY",sum),
    gemiddeld_verkoop = ("UNIT_PRICE","mean")
).reset_index().sort_values(
    by="totaal_verkocht", ascending=False
)

cooking_gear_hvlheid_en_gem_verkoopprijs["gemiddeld_verkoop"] = cooking_gear_hvlheid_en_gem_verkoopprijs["gemiddeld_verkoop"].round(2)


print(cooking_gear_hvlheid_en_gem_verkoopprijs)




                PRODUCT_NAME  totaal_verkocht  gemiddeld_verkoop
9        TrailChef Water Bag            36738               6.20
2              TrailChef Cup            27418               7.16
0          TrailChef Canteen            26544              12.21
6      TrailChef Kitchen Kit            22206              22.85
8         TrailChef Utensils            13776              17.84
4     TrailChef Double Flame            12476             137.54
1         TrailChef Cook Set            10878              52.99
5           TrailChef Kettle             8826              12.42
7     TrailChef Single Flame             8504              65.29
3  TrailChef Deluxe Cook Set             3110             124.18


Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

In [205]:
overizicht_land_stad_count = merge_retailer_site_met_country.groupby("COUNTRY").agg(
    verkoper = ("CITY","first"),
    klanten = ("CITY","nunique")
).reset_index()

print(overizicht_land_stad_count)

#verkoper_df = merge_retailer_site_met_country[["COUNTRY", "CITY"]].drop_duplicates()
#klanten_count = merge_retailer_site_met_country.groupby("COUNTRY")["CITY"].nunique().reset_index(name="klanten")
#overzicht_land_stad_count = verkoper_df.merge(klanten_count, on="COUNTRY", how="left")
#overzicht_land_stad_count = overzicht_land_stad_count.rename(columns={"CITY": "verkoper"})
#print(overzicht_land_stad_count)


           COUNTRY      verkoper  klanten
0        Australia     Melbourne        6
1          Austria          Wien        9
2          Belgium      Zaventem        8
3           Brazil     São Paulo        1
4           Canada      Winnipeg       16
5            China       Beijing        7
6          Denmark     København        5
7          Finland       Tampere        5
8           France         Paris       14
9          Germany     Frankfurt       18
10           Italy       Bologna        9
11           Japan     Osaka-shi        5
12           Korea         Pusan        6
13          Mexico        Cancún        6
14     Netherlands         Beets       15
15           Spain     Barcelona        5
16          Sweden     Stockholm       10
17     Switzerland         Basel        9
18          Taiwan  Chungho City        9
19  United Kingdom        London        9
20   United States      Hartford       62


## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

In [210]:
wel_verkocht = set(dataframes["order_header"]["SALES_STAFF_CODE"].unique())
niet_verkocht = dataframes["sales_staff"][~dataframes["sales_staff"]["SALES_STAFF_CODE"].isin(wel_verkocht)].reset_index()

resultaat_niet_verkocht = niet_verkocht[["FIRST_NAME","LAST_NAME"]]

print(resultaat_niet_verkocht)


   FIRST_NAME      LAST_NAME
0       Denis           Pagé
1        Else         Mörike
2       Frank          Fuchs
3       Fritz         Hirsch
4        Jörg          Kunze
5       Maria      Iacobucci
6        Kick        Kalkman
7       Karin      Bergström
8       Sally          White
9       Frank        Bretton
10    Georges  Saint-Germain
11       Alex      Rodriguez
12   Samantha          Floyd
13      Julie          Olsen
14      Corey         Wright
15    Estelle        Johnson
16       Greg        Belding
17      Julie        Béchard
18      Lucía          Reyna
19     Kazumi        Uragome
20      Alice         Walter
21     Walter         Taylor
22   Penelope        Tamrine
23      Polly         Linton
24      Gracy        Gellens


Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

In [218]:
gem_marge = dataframes["product"]["MARGIN"].mean()
lager_dan_marge = dataframes["product"][dataframes["product"]["MARGIN"] < gem_marge]
aantal_producten_lager = lager_dan_marge.shape[0]
gem_marge_lager = round(lager_dan_marge["MARGIN"].mean(), 2)

overzicht_marges_lager = pandas.DataFrame(
    {"Waarde" : [aantal_producten_lager,gem_marge_lager]},
    index=["Aantal Producten","Gemiddelde Marge"]
)
print(overzicht_marges_lager)



                  Waarde
Aantal Producten   59.00
Gemiddelde Marge    0.29


Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

In [232]:


merge_nieuw = merge_order_details_met_returned_item.merge(
    dataframes["product"],on="PRODUCT_NUMBER"
)
expensive_sales = merge_order_details_met_product[ merge_order_details_met_product["UNIT_SALE_PRICE"] > 500 ]
expensive_products = set(expensive_sales["PRODUCT_NAME"].unique())

returned_merged = dataframes["returned_item"].merge(
    dataframes["order_details"][["ORDER_DETAIL_CODE", "PRODUCT_NUMBER"]],
    on="ORDER_DETAIL_CODE", how="left"
)
returned_merged = returned_merged.merge(
    dataframes["product"][["PRODUCT_NUMBER", "PRODUCT_NAME"]],
    on="PRODUCT_NUMBER", how="left"
)
returned_products = set(returned_merged["PRODUCT_NAME"].unique())

# Stap 4: Bepaal de producten die wel duur verkocht zijn maar nooit zijn teruggebracht
never_returned_products = expensive_products - returned_products

# Stap 5: Maak een overzicht DataFrame (1 kolom, verwacht 13 rijen)
result_df = pandas.DataFrame(list(never_returned_products), columns=["PRODUCT_NAME"])
result_df = result_df.sort_values("PRODUCT_NAME")  # Optioneel, voor nettere output

print(result_df)

                PRODUCT_NAME
0  Hailstorm Steel Woods Set


Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

In [236]:
sales_staff = dataframes["sales_staff"][["LAST_NAME","POSITION_EN"]].copy()

sales_staff["IS_MANAGER"] = ["Ja" if "Manager" in pos else "Nee" for pos in sales_staff["POSITION_EN"]]

resultaat_managers = sales_staff[["LAST_NAME", "IS_MANAGER"]]

print(resultaat_managers)

     LAST_NAME IS_MANAGER
0         Pagé         Ja
1       Michel        Nee
2     Clermont        Nee
3       Jauvin        Nee
4    Wiesinger        Nee
..         ...        ...
97    Laermans        Nee
98     De Crée        Nee
99     Lattrez        Nee
100  Seefelder        Nee
101     Grüner        Nee

[102 rows x 2 columns]


Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [237]:
from datetime import date
date.today().year

2025

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [240]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

2013

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 

In [255]:
sales_staff_jong_of_oud = dataframes["sales_staff"][["LAST_NAME","DATE_HIRED"]].copy()

sales_staff_jong_of_oud["DATE_HIRED"] = pandas.to_datetime(sales_staff_jong_of_oud["DATE_HIRED"], format="%Y-%m-%d")

current_year = date.today().year

sales_staff_jong_of_oud["YEARS_IN_SERVICE"] = current_year - sales_staff_jong_of_oud["DATE_HIRED"].dt.year

sales_staff_jong_of_oud["KORT_IN_DIENST"] = ["Ja" if years < 25 else "Nee" for years in sales_staff_jong_of_oud["YEARS_IN_SERVICE"]]
sales_staff_jong_of_oud["LANG_IN_DIENST"] = ["Ja" if years >= 12 else "Nee" for years in sales_staff_jong_of_oud["YEARS_IN_SERVICE"]]

resultaat_dienstjaren = sales_staff_jong_of_oud[["KORT_IN_DIENST", "LANG_IN_DIENST"]]

print(resultaat_dienstjaren)

    KORT_IN_DIENST LANG_IN_DIENST
0              Nee             Ja
1              Nee             Ja
2              Nee             Ja
3              Nee             Ja
4              Nee             Ja
..             ...            ...
97             Nee             Ja
98             Nee             Ja
99             Nee             Ja
100            Nee             Ja
101            Nee             Ja

[102 rows x 2 columns]


## Van Jupyter Notebook naar Pythonproject

1. Richt de map waarin jullie tot nu toe hebben gewerkt in volgens de mappenstructuur uit de slides.
2. Maak van de ontstane mappenstructuur een Pythonproject dat uitvoerbaar is vanuit de terminal. Maak daarin een .py-bestand dat minstens 5 antwoorden uit dit notebook (in de vorm van een DataFrame) exporteert naar Excelbestanden. Alle notebooks mogen als notebook blijven bestaan.
3. Zorg ervoor dat dit Pythonproject zijn eigen repo heeft op Github. Let op: je virtual environment moet <b><u>niet</u></b> meegaan naar Github.

Je mag tijdens dit proces je uit stap 1 ontstane mappenstructuur aanpassen, zolang je bij het beoordelingsmoment kan verantwoorden wat de motivatie hierachter is. De slides verplichten je dus nergens toe.